# State-of-the-art hierarchical summarization with Huggingface models

In [1]:
import gc
import torch
from tqdm.notebook import tqdm
from utils import SummarizationPipeline, ModelConfig, LoggingConfig, load_all_available_transcripts, TextChunker

/workspace/persistent/ragsum/venv/lib/python3.10/site-packages/_distutils_hack/__init__.py:53: UserWarning: Reliance on distutils from stdlib is deprecated. Users must rely on setuptools to provide the distutils module. Avoid importing distutils or import setuptools first, and avoid setting SETUPTOOLS_USE_DISTUTILS=stdlib. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


In [6]:
transcripts = load_all_available_transcripts()
transcripts.columns

2025-04-30 12:01:17,232 - utils.loaders - INFO - 32307
2025-04-30 12:01:17,322 - utils.loaders - INFO - Successfully loaded local transcripts for 32307
2025-04-30 12:01:17,323 - utils.loaders - INFO - Successfully loaded 32307.csv
2025-04-30 12:01:17,324 - utils.loaders - INFO - 126475
2025-04-30 12:01:17,346 - utils.loaders - INFO - Successfully loaded local transcripts for 126475
2025-04-30 12:01:17,346 - utils.loaders - INFO - Successfully loaded 126475.csv
2025-04-30 12:01:17,347 - utils.loaders - INFO - 26446
2025-04-30 12:01:17,357 - utils.loaders - INFO - Successfully loaded local transcripts for 26446
2025-04-30 12:01:17,357 - utils.loaders - INFO - Successfully loaded 26446.csv
2025-04-30 12:01:17,357 - utils.loaders - INFO - 388904
2025-04-30 12:01:17,375 - utils.loaders - INFO - Successfully loaded local transcripts for 388904
2025-04-30 12:01:17,375 - utils.loaders - INFO - Successfully loaded 388904.csv
2025-04-30 12:01:17,375 - utils.loaders - INFO - 312932093
2025-04-30 

Index(['companyid', 'companyname', 'mostimportantdateutc',
       'mostimportanttimeutc', 'headline', 'full_text', 'uuid', 'word_count',
       'word_count_nltk'],
      dtype='object')

In [2]:
transcripts = load_all_available_transcripts()
transcripts = transcripts[:3]

2025-04-30 11:53:39,499 - utils.loaders - INFO - 32307
2025-04-30 11:53:39,591 - utils.loaders - INFO - Successfully loaded local transcripts for 32307
2025-04-30 11:53:39,592 - utils.loaders - INFO - Successfully loaded 32307.csv
2025-04-30 11:53:39,592 - utils.loaders - INFO - 126475
2025-04-30 11:53:39,615 - utils.loaders - INFO - Successfully loaded local transcripts for 126475
2025-04-30 11:53:39,616 - utils.loaders - INFO - Successfully loaded 126475.csv
2025-04-30 11:53:39,616 - utils.loaders - INFO - 26446
2025-04-30 11:53:39,627 - utils.loaders - INFO - Successfully loaded local transcripts for 26446
2025-04-30 11:53:39,628 - utils.loaders - INFO - Successfully loaded 26446.csv
2025-04-30 11:53:39,629 - utils.loaders - INFO - 388904
2025-04-30 11:53:39,647 - utils.loaders - INFO - Successfully loaded local transcripts for 388904
2025-04-30 11:53:39,647 - utils.loaders - INFO - Successfully loaded 388904.csv
2025-04-30 11:53:39,648 - utils.loaders - INFO - 312932093
2025-04-30 

In [3]:
transcripts = transcripts.full_text.tolist()

In [4]:
checkpoints = ['facebook/bart-large-cnn', 'google-t5/t5-base', 'google/pegasus-x-large', 'human-centered-summarization/financial-summarization-pegasus']

logging_config: LoggingConfig = LoggingConfig()

summaries = []

for checkpoint in checkpoints:

    model_config: ModelConfig = ModelConfig(
        model_name_or_path=checkpoint, device='cuda' if torch.cuda.is_available() else 'cpu'
    )

    pipeline = SummarizationPipeline(model_config=model_config, logging_config=logging_config)
    
    tokenizer = pipeline.get_tokenizer()
    chunker = TextChunker(tokenizer)

    try:
        for transcript in tqdm(transcripts):
            chunks = chunker.chunk_text(transcript)
            chunk_summaries = [pipeline.summarize(chunk) for chunk in chunks]
            combined_summary = " ".join(chunk_summaries)
            max_rounds = 5
            round_count = 0
            final_summary = ""
            while round_count < max_rounds:
                print(f'round {round_count+1}')
                input_ids = tokenizer(combined_summary, return_tensors='pt', truncation=False)['input_ids']
                if input_ids.shape[1] <= min(1024, pipeline.model_max_length):
                    final_summary = combined_summary
                    break
                re_chunks = chunker.chunk_text(combined_summary)
                re_chunk_summaries = [pipeline.summarize(chunk) for chunk in re_chunks]
                combined_summary = " ".join(re_chunk_summaries)
                round_count += 1
            else:
                final_summary = combined_summary
            summaries.append(final_summary)
    finally:
        del pipeline
        del model_config
        torch.cuda.empty_cache()
        gc.collect()

2025-04-30 11:53:39,683 - SummarizationPipeline - INFO - Initializing pipeline with model facebook/bart-large-cnn
2025-04-30 11:53:39,685 - SummarizationPipeline - INFO - Loading tokenizer for facebook/bart-large-cnn


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

round 2


Chunking text: 100%|██████████| 47/47 [00:00<00:00, 684487.11it/s]
2025-04-30 11:59:00,161 - utils.text_chunker - INFO - Text successfully split into 47 chunks.
2025-04-30 11:59:00,161 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-04-30 11:59:00,927 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-04-30 11:59:01,736 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-04-30 11:59:02,522 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-04-30 11:59:03,316 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-04-30 11:59:04,159 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-04-30 11:59:04,983 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-04-30 11:59:05,744 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-04-30 11:59:06,511 - SummarizationPipeline - INFO - Genera

round 1


Chunking text: 100%|██████████| 6/6 [00:00<00:00, 135300.13it/s]
2025-04-30 11:59:38,044 - utils.text_chunker - INFO - Text successfully split into 6 chunks.
2025-04-30 11:59:38,044 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-04-30 11:59:39,020 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-04-30 11:59:39,816 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-04-30 11:59:40,705 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-04-30 11:59:41,472 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-04-30 11:59:42,269 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)


round 2


In [5]:
summaries

['JPMorgan\'s 21st Annual Technology and Automotive Investor Forum is being held at the Consumer Electronics Show. Colette Kress, Executive Vice President and Chief Financial Officer at NVIDIA, will be the first to present. Kress: We are focused on bringing to market a large lineup of new products, particularly in our data center. Oren: FoxConn has decided to join and has adopted our Oren architecture for compute in the many NEVs, not just the NEVs that we are already a part of, but any type of NEV, new energy types of cars across the world. Omniverse has, of course, now been adopted by Mercedes in their next-generation factories similar to some of the other automotive companies. On gaming, in particular, Q2 and Q3 combined, you\'re shipping about 30% below end demand consumption of around $5 billion. We believe that this is just through a combination of the holiday season and a direct focus on making sure working with our AIC partners and our OEMs to move that inventory. Hewlett-Packa